In [3]:
import pandas as pd
from thefuzz import process

# Load transactions file
transactions_path = "C:/Users/lgino/code/Lothardill/Hospitalis/final_countdown.csv"
df_transactions = pd.read_csv(transactions_path, encoding="utf-8-sig")

# Load Amélie file
amelie_path = "C:/Users/lgino/code/Lothardill/Hospitalis/OPENLLP_Produit_Fournisseur.csv"
df_amelie = pd.read_csv(amelie_path, encoding="utf-8-sig")
# Ensure Fournisseur column exists
if "Fournisseur" not in df_transactions.columns:
    raise ValueError("❌ 'Fournisseur' column missing in Transactions file!")


In [5]:
def find_best_match(article_name, choices, threshold=80):
    """
    Find the best fuzzy match for `article_name` within `choices`.
    Returns the matched name if score >= threshold, else None.
    """
    match, score = process.extractOne(article_name, choices)
    return match if score >= threshold else None

In [7]:
df_grouped = df_transactions.groupby("Libellé Article").agg({
    "Fournisseur": "first",  # Keep first supplier per product
    "Commande": "first",     # Keep first order reference
    "Dépôt DMI": "first",    # Keep first depot reference
    "Réf. Article": "first", # Keep first product reference
    "Quantité": "sum",       # Sum total quantity
    "Prix HT": "mean",       # Average unit price (before tax)
    "Prix TTC": "mean",      # Average unit price (tax included)
    "Mnt HT": "sum",         # Total purchase amount (before tax)
    "Mnt TTC": "sum"         # Total purchase amount (tax included)
}).reset_index()

In [ ]:
import time

# Get unique product names from Amélie for fuzzy matching
amelie_products = df_amelie["Étiquettes de lignes"].tolist()

# Create a new column for storing matches
df_transactions["Match"] = None

# Track progress efficiently
total_rows = len(df_transactions)
start_time = time.time()

# Iterate over transactions to find matches
for index, row in df_transactions.iterrows():
    etiquette_match = None

    # 1️⃣ First check if "Fournisseur" appears in "Étiquettes de lignes"
    for etiq in amelie_products:
        if row["Fournisseur"] in etiq:
            etiquette_match = etiq
            break  # Stop once we find a match

    # 2️⃣ If we found a Fournisseur match, try fuzzy matching on "Libellé Article"
    if etiquette_match:
        best_match = find_best_match(row["Libellé Article"], amelie_products)
        df_transactions.at[index, "Match"] = best_match  # Store best match

    # 3️⃣ Lightweight progress tracking
    if index % 500 == 0:  # Update every 500 rows
        elapsed = time.time() - start_time
        print(f"✅ Processed {index}/{total_rows} rows - Elapsed Time: {elapsed:.2f}s")

print("🎉 Matching Completed!")

# Save the updated transactions file
df_transactions.to_csv("C:/Users/lgino/code/Lothardill/Hospitalis/Transactions_with_Match.csv", index=False, encoding="utf-8-sig")
print("✅ Merged file saved successfully!")


✅ Processed 0/5953297 rows - Elapsed Time: 4.57s


In [ ]:
import pandas as pd

# Load Transactions file
transactions_path = "C:/Users/lgino/code/Lothardill/Hospitalis/Transactions_2024.csv"
df_transactions = pd.read_csv(transactions_path, encoding="utf-8-sig")

# Load Amélie file
amelie_path = "/mnt/data/OPENLLP_Produit_Fournisseur.csv"
df_amelie = pd.read_csv(amelie_path, encoding="utf-8-sig")

# Rename columns for clarity
df_amelie.columns = ["Étiquettes de lignes", "Moyenne Prix Unitaire"]

# Show sample data
print("🚀 Transactions Sample:")
print(df_transactions[["Libellé Article", "Prix HT"]].head())

print("\n🚀 Amélie Sample:")
print(df_amelie.head())


In [7]:
import pandas as pd
import numpy as np
from thefuzz import process
from multiprocessing import Pool, cpu_count

# Load data
transactions_path = "C:/Users/lgino/code/Lothardill/Hospitalis/final_countdown.csv"
df_transactions = pd.read_csv("C:/Users/lgino/code/Lothardill/Hospitalis/final_countdown.csv")
amelie_path = "C:/Users/lgino/code/Lothardill/Hospitalis/OPENLLP_Produit_Fournisseur.csv"
df_amelie = pd.read_csv("C:/Users/lgino/code/Lothardill/Hospitalis/OPENLLP_Produit_Fournisseur.csv")

# 1️⃣ Pre-filter Amélie based on Fournisseur to reduce search space
fournisseur_map = {
    row["Étiquettes de lignes"]: row["Moyenne de Prix unitaire (Base remboursement / Quantité)"]
    for _, row in df_amelie.iterrows()
    if any(fournisseur in row["Étiquettes de lignes"] for fournisseur in df_transactions["Fournisseur"].unique())
}
print(f"✅ Fournisseur filtering done! {len(fournisseur_map)} possible matches found.")

# 2️⃣ Fast fuzzy matching function
def match_article(row):
    """Find best matching article using pre-filtered fournisseur list."""
    if row["Fournisseur"] in fournisseur_map:
        best_match, score = process.extractOne(row["Libellé Article"], fournisseur_map.keys())
        return best_match if score >= 80 else None
    return None

# 3️⃣ Parallel processing for ultra-fast matching
def parallel_fuzzy_matching(df):
    """Splits dataframe and applies fuzzy matching in parallel for massive speedup."""
    num_partitions = cpu_count()
    df_split = np.array_split(df, num_partitions)
    with Pool(num_partitions) as pool:
        df = pd.concat(pool.map(lambda x: x.apply(match_article, axis=1), df_split))
    return df

df_transactions["Match"] = parallel_fuzzy_matching(df_transactions)

# 4️⃣ Save optimized result
df_transactions.to_csv("C:/Users/lgino/code/Lothardill/Hospitalis/Transactions_with_Match.csv", index=False, encoding="utf-8-sig")

print("✅ Fully optimized fuzzy matching completed!")

✅ Fournisseur filtering done! 1452 possible matches found.


C:\Users\lgino\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


AttributeError: Can't get local object 'parallel_fuzzy_matching.<locals>.<lambda>'

In [ ]:
# voir ave la columne fournisseur, voir le chiffre pas obliqué des uniques.